In [2]:
import os
import numpy as np

In [11]:
bs = 4
seqlen = 2048
node = 4
warmup = 10
iter = 10
recipes = ['single_device_full', 'single_device_lora', 'distributed_full', 'distributed_lora']

In [13]:
idx = 3
model = 'llama3-8b'
model_dir = '-'.join(['alpaca', model.split('-')[0], 'finetune'])
recipe_log_path = os.path.join(model_dir, 
                               '_'.join([model, recipes[idx], f'bs_{bs}', f'seqlen_{seqlen}', f'node_{node}' if idx>1 else '']).strip('_')+'.log')

f = open(recipe_log_path, 'r')
for line in f.readlines():
    if 'Writing logs' in line:
        log_path = line.split(' ')[-1].strip()
        break

infos = {
    'tokens_per_second_per_gpu': [],
    'peak_memory_active': [],
    'peak_memory_alloc': [],
    'peak_memory_reserved': [],
}

f = open(log_path, 'r')
for idx, line in enumerate(f.readlines()):
    if idx<warmup:
        continue
    if idx>warmup+iter:
        break
    info = line.strip().split(' ')[-4:]
    for data in info:
        infos[data.split(':')[0]].append(float(data.split(':')[1]))

print(recipe_log_path)
for k, v in infos.items():
    if 'token' in k:
        print(f'{k}: {np.mean(v):.0f}')
    else:
        print(f'{k}: {np.max(v):.4f}')


alpaca-llama3-finetune/llama3-8b_distributed_lora_bs_4_seqlen_2048_node_4.log
tokens_per_second_per_gpu: 2000
peak_memory_active: 21.0962
peak_memory_alloc: 21.0962
peak_memory_reserved: 25.4825
